In [ ]:
# -*- coding: utf-8 -*-
"""
This code integrates the "player guessing the computer's number" logic into the 
previous code structure that had both a "computer_guess" and a "player_guess" mode.
The computer guessing functionality remains unchanged.

Fixes made:
- We no longer rely on matching lambda function bytecode to identify prompts. 
  Instead, we store questions and prompts in a fixed order and find the chosen 
  question by function identity.
"""

import sys
import random
import sympy
import inflect

def restart_game():
    restart = check_for_yes_or_no("Would you like to play again? (y/n): ")
    if restart:
        main()
    else:
        print("Thanks for playing!")

def check_for_yes_or_no(question):
    answer = input(question)
    while True:
        if answer.lower() in ['y', 'yes']:
            return True
        elif answer.lower() in ['n', 'no']:
            return False
        else:
            answer = input("Invalid input, please enter yes or no: ")

def check_h_l(question):
    answer = input(question)
    while True:
        if answer.lower() in ['l', 'lower']:
            return "lower"
        elif answer.lower() in ['h', 'higher']:
            return "higher"
        else:
            answer = input("Invalid input, please enter higher or lower: ")

def check_for_guess(number_range):
    if len(number_range) == 1:
        print(f"Okay I've got this! Your number has to be {number_range[0]}.")
        return True
    return False

def is_sum_of_digits_even(num):
    total = sum(int(d) for d in str(num))
    return (total % 2 == 0)

def is_all_digits_unique(num):
    digits = str(num)
    unique_digits = set(digits)
    return len(unique_digits) == len(digits)

def is_even(num):
    return (num % 2 == 0)

def is_divisible_by_3(num):
    return (num % 3 == 0)

def is_divisible_by_5(num):
    return (num % 5 == 0)

def is_multiple_of_10(num):
    return (num % 10 == 0)

def is_greater_or_equal_midpoint(num, number_range):
    midpoint = (number_range[0] + number_range[-1]) // 2
    return (num >= midpoint)

def optimal_binary_question(number_range, question_list):
    """
    Identify the best binary question to ask from the provided question_list.
    question_list is a list of tuples: (function, prompt)
    Each function is a lambda that returns True/False.
    """
    best_question = None
    best_split = float('inf')  # Start with a large number

    for question_func, _ in question_list:
        yes_subset = [num for num in number_range if question_func(num)]
        no_subset = [num for num in number_range if not question_func(num)]

        largest_subset = max(len(yes_subset), len(no_subset))
        total = len(number_range)
        size_yes = len(yes_subset)
        size_no = len(no_subset)
        expected_size = (size_yes / total) * size_yes + (size_no / total) * size_no

        if largest_subset < best_split or (largest_subset == best_split and expected_size < best_split):
            best_split = largest_subset
            best_question = question_func

    return best_question

def computer_guess():
    print("Think of a whole number between 1-1000 and write it down somewhere safe where I can't see.")
    print("If you don't stay within those parameters I won't be able to guess your number :(")

    number_range = list(range(1, 1001))

    # We store the questions and prompts in a list of tuples
    # The prompt for the midpoint question will be dynamically adjusted.
    # Note: We do not rely on comparing code objects. We return the chosen function and 
    # then find it in this list by identity.
    question_list = [
        (lambda num: (sum(int(d) for d in str(num)) % 2 == 0), "Is the sum of the digits of your number even? (y/n): "),
        (lambda num: (len(set(str(num))) == len(str(num))),    "Are all the digits of your number unique? (y/n): "),
        (lambda num: num % 2 == 0,                             "Is your number even? (y/n): "),
        (lambda num: num % 3 == 0,                             "Is your number divisible by 3? (y/n): "),
        (lambda num: num % 5 == 0,                             "Is your number divisible by 5? (y/n): "),
        (lambda num: num % 10 == 0,                            "Is your number a multiple of 10? (y/n): "),
        (lambda num: num >= (number_range[0] + number_range[-1]) // 2, "Is your number greater than or equal to the midpoint? (y/n): ")
    ]

    while len(number_range) > 1:
        question = optimal_binary_question(number_range, question_list)

        # Find the corresponding prompt
        for q_func, prompt in question_list:
            if q_func is question:
                # If it's the midpoint question, dynamically insert the midpoint value
                if "midpoint" in prompt:
                    midpoint = (number_range[0] + number_range[-1]) // 2
                    prompt = f"Is your number greater than or equal to {midpoint}? (y/n): "
                response = check_for_yes_or_no(prompt)
                break
        else:
            raise ValueError("Selected question does not match any known prompt.")

        # Filter based on the response
        number_range = [num for num in number_range if question(num) == response]

        # Debug range if small
        if len(number_range) <= 10:
            print(f"DEBUG: Current number range: {number_range}")

        if check_for_guess(number_range):
            return

    # Final guess
    if len(number_range) == 1:
        print(f"Your number is {number_range[0]}!")
    else:
        print("I couldn't guess your number with the given questions.")

def player_guess():
    points = 100
    x = random.randint(1, 1001)

    c_1 = 0
    c_2 = 0
    c_3 = 0
    c_4 = 0
    c_5 = 0
    c_6 = 0
    c_7 = 0
    c_8 = 0
    c_9 = 0
    c_10 = 0

    def guess():
        nonlocal points
        a_1 = input("Do you have a guess?:")
        if a_1.lower() in ['yes', 'y', 'yeah', 'yep']:
            g_1 = input("What is your guess?:")
            if g_1 == str(x):
                print("Correct! The number is", x)
                print("You won with", points, "points remaining!")
                return True
            else:
                print("Sorry, that's incorrect")
                return False
        elif a_1.lower() in ['no', 'n', 'nah', 'nope']:
            print("Then let us proceed")
            return False
        else:
            # If the user directly enters a guess or something else
            if a_1 == str(x):
                print("Correct! The number is:", x)
                print("You won with", points, "points remaining!")
                return True
            else:
                print("Sorry, either your guess was wrong or I did not understand your input.")
                return False

    def first_letter_of_number(number):
        i = inflect.engine()
        word = i.number_to_words(number)
        print("The first letter is", word[0])

    def parity(val):
        p = val % 2
        if p == 0:
            print('The number is even')
        else:
            print("The number is odd")

    def digits(val):
        print("There are", len(str(val)), "digits in the number")

    def sum_of_last_2(val):
        s_val = str(val)
        if len(s_val) == 1:
            s = int(s_val)
        else:
            last_two = s_val[-2:]
            s = sum(int(d) for d in last_two)
        print("The sum of the final two digits in the number is", s)

    def first_parity(val):
        f = int(str(val)[0]) % 2
        if f == 0:
            print('The first digit of the number is even')
        else:
            print('The first digit of the number is odd')

    def product_two(val):
        s_val = str(val)
        if len(s_val) == 1:
            p = int(s_val)
        else:
            d = int(s_val[-2])
            g = int(s_val[-1])
            p = d * g
        print("The product of the final two digits in the number is", p)

    def half_check(val):
        if val >= 500:
            print("The number is greater or equal to 500")
        else:
            print("The number is less than 500")

    def higher_lower(val):
        e = input("What number would you like to check? You only get one, so choose wisely!:")
        try:
            e = int(e)
            if val < e:
                print("The number is less than", e)
            elif val > e:
                print("The number is greater than", e)
            else:
                print("The number is equal to", e)
        except ValueError:
            print("Invalid input. Please enter a numeric value next time.")

    def prime_check(val):
        if sympy.isprime(val):
            print("The number is prime")
        else:
            print("The number is not prime")

    def last_digit(val):
        v = str(val)[-1]
        print("The last digit of the number is", v)

    def point_check(val):
        nonlocal points
        if points <= 0:
            y = input("You ran out of points! Would you like to make a final guess?:")
            if y.lower() in ['yes', 'y']:
                final_guess = input("What is your final guess?:")
                if final_guess == str(val):
                    print("Correct! The answer was", val)
                else:
                    print("Sorry, that's not correct... The number was", val, "better luck next time!")
            else:
                print("Better luck next time! The number was", val)
            return True
        return False

    def pointy():
        print("Points remaining =", points)

    def f(q):
        def ask_again():
            nonlocal c_1, c_2, c_3, c_4, c_5, c_6, c_7, c_8, c_9, c_10, points
            q_new = input("Could you please try again with a new number (1-10)?:")
            handle_question_choice(q_new)

        def handle_question_choice(question):
            nonlocal c_1, c_2, c_3, c_4, c_5, c_6, c_7, c_8, c_9, c_10, points
            if question == '1':
                if c_1 == 0:
                    first_letter_of_number(x)
                    c_1 = 1
                    points -= 20
                    if guess():
                        return True
                else:
                    print("You have already tried question 1.")
                    ask_again()
            elif question == '2':
                if c_2 == 0:
                    parity(x)
                    c_2 = 1
                    points -= 10
                    if guess():
                        return True
                else:
                    print("You have already tried question 2.")
                    ask_again()
            elif question == '3':
                if c_3 == 0:
                    digits(x)
                    c_3 = 1
                    points -= 20
                    if guess():
                        return True
                else:
                    print("You have already tried question 3.")
                    ask_again()
            elif question == '4':
                if c_4 == 0:
                    sum_of_last_2(x)
                    c_4 = 1
                    points -= 20
                    if guess():
                        return True
                else:
                    print("You have already tried question 4.")
                    ask_again()
            elif question == '5':
                if c_5 == 0:
                    first_parity(x)
                    c_5 = 1
                    points -= 10
                    if guess():
                        return True
                else:
                    print("You have already tried question 5.")
                    ask_again()
            elif question == '6':
                if c_6 == 0:
                    product_two(x)
                    c_6 = 1
                    points -= 20
                    if guess():
                        return True
                else:
                    print("You have already tried question 6.")
                    ask_again()
            elif question == '7':
                if c_7 == 0:
                    half_check(x)
                    points -= 10
                    c_7 = 1
                    if guess():
                        return True
                else:
                    print("You have already tried question 7.")
                    ask_again()
            elif question == '8':
                if c_8 == 0:
                    higher_lower(x)
                    points -= 10
                    c_8 = 1
                    if guess():
                        return True
                else:
                    print("You have already tried question 8.")
                    ask_again()
            elif question == '9':
                if c_9 == 0:
                    prime_check(x)
                    c_9 = 1
                    points -= 10
                    if guess():
                        return True
                else:
                    print("You have already tried question 9.")
                    ask_again()
            elif question == '10':
                if c_10 == 0:
                    last_digit(x)
                    c_10 = 1
                    points -= 20
                    if guess():
                        return True
                else:
                    print("You have already tried question 10.")
                    ask_again()
            else:
                print("Invalid choice.")
                ask_again()
            return False

        return handle_question_choice(q)

    # Start the player guessing game
    print("Welcome to the game! The rules are simple: the computer has picked a number between 1-1000 and you must guess it!")
    print("You have a set of predetermined questions and a budget of 100 points. Each question reduces your points. If you run out of points, game over!")
    print("Your questions are:")
    print("1 - What letter does your number start with? (Cost - 20)") # not binary
    print("2 - Is your number odd or even? (Cost - 10)") # binary
    print("3 - How many digits are in your number? (Cost - 20)") # not binary
    print("4 - What is the sum of the last two digits of your number? (Cost - 20)") # not binary
    print("5 - Is the first digit in your number odd or even? (Cost - 10)") # binary
    print("6 - What is the product of the last two digits of your number? (Cost - 20)") # not binary
    print("7 - Is the number more or less than 500? (Cost - 10)") # binary
    print("8 - Is the number higher or lower than x? (Cost - 10)") # binary
    print("9 - Is the number prime? (Cost - 10)") # binary
    print("10 - What is the last digit in the number? (Cost - 20)") # not binary

    asked_questions = []

    def ask_question():
        nonlocal asked_questions, points
        q = input("Choose a question from 1-10:")
        while q in asked_questions:
            q = input("You've already asked that! Please select a new question!:")
        asked_questions.append(q)
        if f(q):
            # They guessed the number correctly inside f(q)/guess()
            return True
        pointy()
        if point_check(x):
            # If points ran out and final guess sequence ended
            return True
        return False

    # Ask up to 10 questions or until the player wins or runs out of points
    for _ in range(10):
        if ask_question():
            # Player either won or game ended
            return

    # If we reach here, the player asked 10 questions and didn't guess correctly or run out of points
    final_try = input("You've used all your allowed questions. Make a final guess?:")
    if final_try.lower() in ['yes', 'y']:
        final_guess = input("Final guess, what's the number?:")
        if final_guess == str(x):
            print("Correct! The number was", x)
        else:
            print("Nope, sorry! The number was", x)
    else:
        print("No final guess? The number was", x)

def main():
    role = input("Welcome player! This is the guessing game. Enter 'player' for player to guess or 'computer' for the computer to guess: ")

    while role.lower() not in ['player', 'computer']:
        role = input("Invalid choice. Please enter 'player' or 'computer': ")

    print(f"Excellent! The person guessing will be the {role.lower()}.")

    if role.lower() == "computer":
        computer_guess()
    else:
        player_guess()

    restart_game()

main()


Welcome player! This is the guessing game. Enter 'player' for player to guess or 'computer' for the computer to guess:  computer


Excellent! The person guessing will be the computer.
Think of a whole number between 1-1000 and write it down somewhere safe where I can't see.
If you don't stay within those parameters I won't be able to guess your number :(


Is your number even? (y/n):  n
Is the sum of the digits of your number even? (y/n):  y
Is your number greater than or equal to 500? (y/n):  n
Is your number greater than or equal to 255? (y/n):  n
Is your number greater than or equal to 132? (y/n):  y
Is your number greater than or equal to 197? (y/n):  y
Is your number greater than or equal to 232? (y/n):  n


DEBUG: Current number range: [211, 213, 215, 217, 219, 231]


Is your number divisible by 3? (y/n):  n


...

DEBUG: Current number range: [440, 442, 444, 446, 448]


Is your number greater than or equal to 444? (y/n):  y


DEBUG: Current number range: [444, 446, 448]


Is your number greater than or equal to 446? (y/n):  n


DEBUG: Current number range: [444]
Okay I've got this! Your number has to be 444.
